# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [ ]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
# sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [61]:
import pandas as pd

df = pd.read_csv('USERS_WALLET.csv')
users=df.copy()
print(users)

In [60]:
users.columns

## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [59]:
!pip install openai

In [4]:
# Usar a openai API é uma maneira de simular de maneira simpleficada um modelo focado para analise sentimental/quatitativa da carteira do usuário

openai_api_key = 'API'

In [57]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(stocks):
  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em investimentos."
      },
      {
          "role": "user",
          "content": f"Crie uma análise de mercado da carteira de ações com a seguinte composição: {stocks}"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

users_new =[]
id=0
stocks=1
for index, row in users.iterrows():
  news = generate_ai_news(row[stocks])
  users_new.append({
      'UserID': row[id],
      'Stocks': row[stocks],
      'MarketOverview': news
  })

## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [58]:
users.to_csv()